<a href="https://colab.research.google.com/github/abnermassimo1/Metodos-Numericos/blob/main/Euler_y_Taylor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Método de Euler y Método de Taylor de orden n

### Método de Euler

El método de Euler es una técnica numérica de un solo paso para aproximar la solución del problema de valor inicial:

\[
\begin{cases}
y' = f(t, y), \\
y(a) = \alpha,
\end{cases}
\]

Se utiliza una partición del intervalo \([a, b]\) en \(N\) subintervalos de tamaño \(h = \frac{b - a}{N}\), generando los nodos \(t_i = a + ih\), para \(i = 0, 1, \dots, N\).

La fórmula de recurrencia del método de Euler es:

$$
\[
w_{i+1} = w_i + h f(t_i, w_i),
\]
$$


donde \(w_i\) es la aproximación de la solución \(y(t_i)\). Este método es de primer orden y su error global es del orden \(O(h)\).

---

### Método de Taylor de orden \(n\)

El método de Taylor de orden \(n\) mejora la precisión al incluir derivadas superiores de \(y(t)\), calculadas a partir de la ecuación diferencial original. A partir del mismo problema inicial:

\[
\begin{cases}
y' = f(t, y), \\
y(a) = \alpha,
\end{cases}
\]

la aproximación se realiza usando una expansión en serie de Taylor alrededor de \(t_i\):


$$
\[
w_{i+1} = w_i + \sum_{k=1}^{n} \frac{h^k}{k!} y^{(k)}(t_i),
\]
$$


donde \(y^{(k)}(t_i)\) representa la \(k\)-ésima derivada total de \(y(t)\) evaluada en \(t_i\), obtenida simbólicamente por diferenciación sucesiva de \(f(t, y)\).

Este método tiene un error global del orden \(O(h^n)\), y se vuelve más preciso a medida que se incrementa \(n\), aunque también requiere mayor complejidad en el cálculo de derivadas.



In [2]:
import numpy as np
import pandas as pd
import sympy as sp

In [20]:
# Método de Euler
def euler_method(f, a, b, alpha, N, y_exact=None):

    h = (b - a) / N                # Paso h
    t = a                         # Valor inicial de t
    w = alpha                     # Valor inicial w ≈ y
    data = [[t, w, y_exact(t), abs(y_exact(t) - w)] if y_exact else [t, w]]

    # Iteración de Euler
    for i in range(1, N + 1):
        w = w + h * f(t, w)       # Paso de Euler
        t = a + i * h             # Avanza t
        if y_exact:
            exact = y_exact(t)
            error = abs(exact - w)
            data.append([t, w, exact, error])
        else:
            data.append([t, w])

    # Crear tabla
    columnas = ['t_i', 'w_i']
    if y_exact:
        columnas += ['y(t_i)', '|y(t_i) - w_i|']
    tabla = pd.DataFrame(data, columns=columnas)

    return tabla

# EJEMPLO:
# Definimos la derivada y' = f(t, y) y la solución exacta (si se conoce)
t, y = sp.symbols('t y')
f_expr = y - t**2 + 1                             # Ecuación: y' = y - t^2 + 1
f_lambd = sp.lambdify((t, y), f_expr, modules='numpy')  # Convertimos a función evaluable

# Solución exacta conocida: y(t) = (t+1)^2 - 0.5*exp(t)
y_exact_expr = (t + 1)**2 - 0.5 * sp.exp(t)
y_exact_func = sp.lambdify(t, y_exact_expr, modules='numpy')

# Parámetros del problema
a = 0         # Inicio del intervalo
b = 2         # Fin del intervalo
alpha = 0.5   # Condición inicial: y(0) = 0.5
N = 10        # Número de subintervalos

# Ejecutamos el método de Euler
tabla_resultado = euler_method(f_lambd, a, b, alpha, N, y_exact=y_exact_func)

# Mostramos la tabla
print(tabla_resultado.to_string(index=False))


 t_i      w_i   y(t_i)  |y(t_i) - w_i|
 0.0 0.500000 0.500000        0.000000
 0.2 0.800000 0.829299        0.029299
 0.4 1.152000 1.214088        0.062088
 0.6 1.550400 1.648941        0.098541
 0.8 1.988480 2.127230        0.138750
 1.0 2.458176 2.640859        0.182683
 1.2 2.949811 3.179942        0.230130
 1.4 3.451773 3.732400        0.280627
 1.6 3.950128 4.283484        0.333356
 1.8 4.428154 4.815176        0.387023
 2.0 4.865785 5.305472        0.439687


¿Qué hace el código?
Usa SymPy para derivar simbólicamente
𝑦
′
,
𝑦
′
′
,
𝑦
′
′
′
,
.
.
.
y
′
 ,y
′′
 ,y
′′′
 ,... hasta orden
𝑛
n.

Evalúa esas derivadas y construye la fórmula de Taylor numéricamente.

Genera una tabla con:

𝑡
𝑖
t
i
​
 : nodos

𝑤
𝑖
w
i
​
 : aproximación por Taylor

𝑦
(
𝑡
𝑖
)
y(t
i
​
 ): valor exacto

Error absoluto
∣
𝑦
(
𝑡
𝑖
)
−
𝑤
𝑖
∣
∣y(t
i
​
 )−w
i
​
 ∣

In [24]:
import numpy as np
import pandas as pd
import sympy as sp
import math # Import the math module

def metodo_taylor_n(f_expr, a, b, alpha, N, n, y_exacta=None):

    # Variables simbólicas
    t, y = sp.symbols('t y')
    h = (b - a) / N
    ti = a
    wi = alpha

    # Lista de derivadas sucesivas: y', y'', y''', ...
    derivadas = [f_expr]
    for i in range(1, n):
        anterior = derivadas[-1]
        total_diff = sp.diff(anterior, t) + sp.diff(anterior, y) * f_expr
        derivadas.append(total_diff)

    # Convertir derivadas a funciones evaluables numéricamente
    derivadas_eval = [sp.lambdify((t, y), der, modules='numpy') for der in derivadas]

    # Lista de resultados
    resultados = []

    for i in range(N + 1):
        # Evaluación exacta y error (si hay)
        y_val = y_exacta(ti) if y_exacta else None
        error = abs(wi - y_val) if y_val is not None else None

        if y_exacta:
            resultados.append([ti, wi, y_val, error])
        else:
            resultados.append([ti, wi])

        # Último paso, no se avanza más
        if i == N:
            break

        # Cálculo de siguiente wi con fórmula de Taylor de orden n
        suma = 0
        for k in range(n):  # desde k = 0 hasta n-1 representa derivadas de orden k+1
            deriv_k = derivadas_eval[k](ti, wi)
            # Use math.factorial instead of np.math.factorial
            suma += (h**(k + 1) / math.factorial(k + 1)) * deriv_k

        wi = wi + suma
        ti = a + (i + 1) * h

    # Crear tabla
    columnas = ['t_i', 'w_i']
    if y_exacta:
        columnas += ['y(t_i)', '|y(t_i) - w_i|']
    tabla = pd.DataFrame(resultados, columns=columnas)

    return tabla

# -------- EJEMPLO --------
t, y = sp.symbols('t y')
f_expr = y - t**2 + 1

# Solución exacta: y(t) = (t+1)**2 - 0.5*sp.exp(t)
y_exact_expr = (t + 1)**2 - 0.5 * sp.exp(t)
y_exact_func = sp.lambdify(t, y_exact_expr, modules='numpy')

# Parámetros del problema
a = 0
b = 2
alpha = 0.5
N = 10
orden = 4  # Método de Taylor de orden 4

# Ejecutar el método
tabla = metodo_taylor_n(f_expr, a, b, alpha, N, orden, y_exacta=y_exact_func)

# Mostrar resultados
print(tabla.to_string(index=False))

 t_i      w_i   y(t_i)  |y(t_i) - w_i|
 0.0 0.500000 0.500000        0.000000
 0.2 0.829300 0.829299        0.000001
 0.4 1.214091 1.214088        0.000003
 0.6 1.648947 1.648941        0.000006
 0.8 2.127240 2.127230        0.000010
 1.0 2.640874 2.640859        0.000015
 1.2 3.179964 3.179942        0.000022
 1.4 3.732432 3.732400        0.000032
 1.6 4.283529 4.283484        0.000045
 1.8 4.815238 4.815176        0.000061
 2.0 5.305555 5.305472        0.000083
